In [ ]:
# import joblib  # Better alternative for large objects
# import numpy as np

# granger_results_lstm = {}

# for company, results in stationary_columns.items():
#     print(f"Processing company: {company}")
#     df_company = companies_data_daily_final_full[companies_data_daily_final_full['company'] == company].copy()
    
#     # Ensure chronological ordering
#     df_company = df_company.sort_index()  # Critical for time series
    
#     granger_results_lstm[company] = {}
    
#     for stock_var in results['STOCK']:
#         granger_results_lstm[company][stock_var] = {}
#         print(f"  Stock variable: {stock_var}")
        
#         for twitter_var in results['TWITTER']:
#             print(f"Twitter variable: {twitter_var}")
#             max_lags = granger_results.get(company, {}).get(stock_var, {}).get(twitter_var, {}).get('optimal_lags', 15)
#             max_lags = int(max_lags)
#             print(f"Max lags: {max_lags}")
#             # Create dataset with correct column order [STOCK, TWEET]
#             data = df_company[[stock_var, twitter_var]].copy()
            
            
#             # Temporal split (preserve order)
#             n = len(data)
#             train_size = int(0.6 * n)
#             valid_size = int(0.2 * n)
            
          
            
#             # Standardize using TRAIN stats only
#             scaler = StandardScaler()
#             data_train_scaled = scaler.fit_transform(data_train)
#             data_valid_scaled = scaler.transform(data_valid)
#             data_test_scaled = scaler.transform(data_test)
            
#             # Run causality test
#             try:
#                 results_cas = nonlincausality.nonlincausalityNN(
#                 x=data_train_scaled,
#                 maxlag=max_lags,  # Reduced maxlag for small datasets
#                 NN_config=['d', 'dr', 'd'],  # Added extra Dense layer for better feature extraction
#                 NN_neurons=[8, 0.3, 4],  # Reduced neurons to prevent overfitting
#                 x_test=data_test_scaled,
#                 run=3,  # Reduced runs for faster computation
#                 epochs_num=[30, 20],  # Reduced epochs
#                 learning_rate=[0.005, 0.0005],  # Adjusted learning rates
#                 batch_size_num=16,  # Increased batch size for stability
#                 x_val=data_valid_scaled,
#                 regularization='l1_l2',  # Combined regularization
#                 reg_alpha=[0.005, 0.005],  # Smaller regularization
#                 callbacks=None,
#                 verbose=False,
#                 plot=False
#                 )
#                 # Store lightweight results summary
#                 granger_results_lstm[company][stock_var][twitter_var] = {
#                     'lags': {},
#                     'metadata': {
#                         'train_start': data_train.index[0],
#                         'train_end': data_train.index[-1],
#                         'test_start': data_test.index[0],
#                         'test_end': data_test.index[-1],
#                         'n_obs': {
#                             'train': len(data_train),
#                             'valid': len(data_valid),
#                             'test': len(data_test)
#                         }
#                     }
#                 }
                
#                 for lag, res_obj in results_cas.items():
#                     granger_results_lstm[company][stock_var][twitter_var]['lags'][lag] = {
#                         'p_value': res_obj.p_value,
#                         'test_statistic': res_obj.test_statistic,
#                         'RSS_X': res_obj.RSS_X_all,
#                         'RSS_XY': res_obj.RSS_XY_all,
#                         'best_error_X': res_obj.best_errors_X.tolist(),  # Convert arrays
#                         'best_error_XY': res_obj.best_errors_XY.tolist()
#                     }
                
#             except Exception as e:
#                 print(f"Failed for {company}-{stock_var}-{twitter_var}: {str(e)}")
#                 granger_results_lstm[company][stock_var][twitter_var] = {'error': str(e)}

# # Save results with metadata preservation
# joblib.dump(granger_results_lstm, 'granger_results_lstm.pkl')

In [ ]:
def tune_lstm_hyperparameters(X_train, y_train, input_shape):
    """Find optimal units and epochs using time-series cross-validation"""
    best_units = 32  # Default
    best_epochs = 100  # Default
    best_val_loss = float('inf')
    
    # Hyperparameter search space
    units_options = [16, 32, 48]
    epochs_options = [10, 30, 60]
    
    # Reduced CV for efficiency
    tscv = TimeSeriesSplit(n_splits=2)
    
    for units in units_options:
        for max_epochs in epochs_options:
            val_losses = []
            
            for train_idx, val_idx in tscv.split(X_train):
                # Split training/validation
                X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
                y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
                
                # Build model
                model = Sequential([
                    LSTM(units, input_shape=input_shape),
                    Dense(1)
                ])
                model.compile(optimizer='adam', loss='mse')
                
                # Train with early stopping
                early_stop = EarlyStopping(monitor='val_loss', 
                                          patience=5, 
                                          restore_best_weights=True,
                                          verbose=0)
                
                history = model.fit(
                    X_fold_train, y_fold_train,
                    validation_data=(X_fold_val, y_fold_val),
                    epochs=max_epochs,
                    batch_size=32,
                    callbacks=[early_stop],
                    verbose=0
                )
                
                # Track best validation loss
                val_losses.append(min(history.history['val_loss']))
            
            # Average validation loss across folds
            mean_val_loss = np.mean(val_losses)
            if mean_val_loss < best_val_loss:
                best_val_loss = mean_val_loss
                best_units = units
                best_epochs = max_epochs
    
    return best_units, best_epochs

def train_final_model(X_train, y_train, input_shape, units, epochs):
    """Train final model with optimal hyperparameters"""
    model = Sequential([
        LSTM(units, input_shape=input_shape),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    
    # Early stopping on validation split
    early_stop = EarlyStopping(monitor='val_loss', 
                              patience=5, 
                              restore_best_weights=True,
                              verbose=0)
    
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stop],
        verbose=0
    )
    return model

def test_granger_causality(Y, X_base, df, n_lags=5, test_size=0.2, 
                          n_bootstrap=200, block_size=5):
    """
    Main Granger causality test function with:
    - Hyperparameter tuning
    - Block bootstrapping
    """
    # 1. Prepare data
    temp_df = pd.DataFrame(index=df.index)
    temp_df[Y] = df[Y]
    # Create Y lags
    for i in range(1, n_lags+1):
        temp_df[f'{Y}_lag_{i}'] = temp_df[Y].shift(i)
    
    # Identify X lags
    X_lags = [f'{X_base}_lag_{i}' for i in range(1, n_lags+1)]
    missing = [col for col in X_lags if col not in df.columns]
    if missing:
        print(f"Missing columns for {X_base}: {missing}")
        return None, None, False, None
    
    # Combine and clean
    data = pd.concat([temp_df, df[X_lags]], axis=1).dropna()
    y_data = data[Y].values
    Y_lag_cols = [f'{Y}_lag_{i}' for i in range(1, n_lags+1)]
    full_index = data.index
    
    # 2. Train-test split (time-based)
    n_test = int(len(data) * test_size)
    n_train = len(data) - n_test
    splits = {
        'train': slice(None, n_train),
        'test': slice(n_train, None)
    }
    
    # 3. Prepare feature sets
    # Restricted model (only Y lags)
    X_restricted = data[Y_lag_cols].values
    
    # Unrestricted model (Y and X lags)
    X_unrestricted = np.zeros((len(data), n_lags, 2))
    for i in range(len(data)):
        for j in range(n_lags):
            X_unrestricted[i, j, 0] = data[Y_lag_cols[j]].iloc[i]
            X_unrestricted[i, j, 1] = data[X_lags[j]].iloc[i]
    
    # Split data
    X_res_train, X_res_test = X_restricted[splits['train']], X_restricted[splits['test']]
    X_unres_train, X_unres_test = X_unrestricted[splits['train']], X_unrestricted[splits['test']]
    y_train, y_test = y_data[splits['train']], y_data[splits['test']]
    
    # 4. Scaling
    scaler_res = StandardScaler()
    X_res_train_scaled = scaler_res.fit_transform(X_res_train)
    X_res_test_scaled = scaler_res.transform(X_res_test)
    X_res_train_3d = X_res_train_scaled.reshape((-1, n_lags, 1))
    X_res_test_3d = X_res_test_scaled.reshape((-1, n_lags, 1))
    
    scaler_unres = StandardScaler()
    X_unres_train_2d = X_unres_train.reshape((-1, n_lags*2))
    X_unres_test_2d = X_unres_test.reshape((-1, n_lags*2))
    X_unres_train_scaled = scaler_unres.fit_transform(X_unres_train_2d)
    X_unres_test_scaled = scaler_unres.transform(X_unres_test_2d)
    X_unres_train_3d = X_unres_train_scaled.reshape((-1, n_lags, 2))
    X_unres_test_3d = X_unres_test_scaled.reshape((-1, n_lags, 2))
    
    # 5. Hyperparameter tuning
    print(f"Tuning hyperparameters for {Y} <- {X_base}...")
    units_res, epochs_res = tune_lstm_hyperparameters(
        X_res_train_3d, y_train, (n_lags, 1))
    units_unres, epochs_unres = tune_lstm_hyperparameters(
        X_unres_train_3d, y_train, (n_lags, 2))
    
    print(f"Best params - Restricted: units={units_res}, epochs={epochs_res}")
    print(f"Best params - Unrestricted: units={units_unres}, epochs={epochs_unres}")
    
    # 6. Train final models
    model_res = train_final_model(
        X_res_train_3d, y_train, (n_lags, 1), units_res, epochs_res)
    model_unres = train_final_model(
        X_unres_train_3d, y_train, (n_lags, 2), units_unres, epochs_unres)
    
    # 7. Original predictions
    pred_res = model_res.predict(X_res_test_3d).flatten()
    pred_unres = model_unres.predict(X_unres_test_3d).flatten()
    e_res = y_test - pred_res
    e_unres = y_test - pred_unres
    d_original = e_res**2 - e_unres**2
    d_bar_original = np.mean(d_original)
    
    # 8. Bootstrapping
    base_series = df.loc[full_index, X_base].copy()
    bootstrap_stats = []
    
    print(f"Running {n_bootstrap} bootstraps...")
    for b in range(n_bootstrap):
        if (b+1) % 50 == 0:
            print(f"  Bootstrap {b+1}/{n_bootstrap}")
        
        # Create shuffled X series
        shuffled_base = block_shuffle(base_series, block_size)
        
        # Create new lag features from shuffled series
        temp = pd.DataFrame(index=full_index)
        temp[X_base] = shuffled_base
        for i in range(1, n_lags+1):
            temp[f'{X_base}_lag_{i}'] = temp[X_base].shift(i)
        
        # Build new unrestricted dataset
        X_unrestricted_shuffled = np.zeros((len(data), n_lags, 2))
        for i in range(len(data)):
            for j in range(n_lags):
                X_unrestricted_shuffled[i, j, 0] = data[Y_lag_cols[j]].iloc[i]
                X_unrestricted_shuffled[i, j, 1] = temp[X_lags[j]].iloc[i]
        
        # Extract test portion
        X_unres_test_shuffled = X_unrestricted_shuffled[splits['test']]
        X_unres_test_shuffled_2d = X_unres_test_shuffled.reshape((-1, n_lags*2))
        X_unres_test_shuffled_scaled = scaler_unres.transform(X_unres_test_shuffled_2d)
        X_unres_test_shuffled_3d = X_unres_test_shuffled_scaled.reshape((-1, n_lags, 2))
        
        # Predict with shuffled X
        pred_unres_shuffled = model_unres.predict(X_unres_test_shuffled_3d).flatten()
        e_unres_shuffled = y_test - pred_unres_shuffled
        
        # Compute loss difference
        d_shuffled = e_res**2 - e_unres_shuffled**2
        d_bar_shuffled = np.mean(d_shuffled)
        bootstrap_stats.append(d_bar_shuffled)
    
    # 9. Calculate p-value
    bootstrap_stats = np.array(bootstrap_stats)
    p_value = (np.sum(bootstrap_stats >= d_bar_original) + 1) / (n_bootstrap + 1)
    causal = (d_bar_original > 0) and (p_value < 0.05)
    
    return {
        'Y': Y,
        'X': X_base,
        'd_bar': d_bar_original,
        'p_value': p_value,
        'causal': causal,
        'units_res': units_res,
        'epochs_res': epochs_res,
        'units_unres': units_unres,
        'epochs_unres': epochs_unres,
        'bootstrap_mean': np.mean(bootstrap_stats),
        'boostrap_stats':bootstrap_stats
    }

In [ ]:
def create_rolling_predictions(df, tweet_cols, stock_cols, test_days=100, lookback=7):
   # Split into train and test
   train_df = df[:-test_days]
   test_df = df[-test_days:]
   
   predictions = []
   
   for i in range(len(test_df)):
       # Get updated training data including latest actual values
       current_train = pd.concat([train_df, test_df[:i]])
       
       # Prepare data
       X, y, scaler_y = prepare_data(current_train, tweet_cols, stock_cols, lookback)
       
       # Train model
       model = create_model((lookback, len(tweet_cols)), len(stock_cols))
       model.fit(X, y, epochs=50, batch_size=32, verbose=0)
       
       # Predict next day
       last_sequence = scaler_X.transform(current_train[tweet_cols].tail(lookback))
       pred = model.predict(last_sequence.reshape(1, lookback, -1))
       pred = scaler_y.inverse_transform(pred)[0]
       predictions.append(pred)
   
   return np.array(predictions)

predictions = create_rolling_predictions(df, tweet_columns, stock_columns)

In [ ]:
def evaluate_lookbacks(df, tweet_cols, stock_cols, lookbacks=[1]):
    results = {}
    
    for lookback in lookbacks:
        X, y, scaler_y = prepare_data(df, tweet_cols, stock_cols, lookback)
        
        # Train/test split
        split = int(0.8 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]
        
        # Create and train model
        model = create_model((lookback, len(tweet_cols)), len(stock_cols))
        history = model.fit(X_train, y_train, epochs=50, batch_size=32, 
                          validation_split=0.1, verbose=0)
        
        # Evaluate
        predictions = model.predict(X_test)
        predictions = scaler_y.inverse_transform(predictions)
        true_values = scaler_y.inverse_transform(y_test)
        
        mse = np.mean((predictions - true_values) ** 2)
        results[lookback] = {
            'mse': mse,
            'val_loss': min(history.history['val_loss'])
        }
    
    return results



In [ ]:
results = evaluate_lookbacks(merged_daily, tweet_columns, stock_columns)

In [ ]:


def block_shuffle(series, block_size):
    """Shuffle blocks of a time series while preserving internal structure."""
    n = len(series)
    n_blocks = n // block_size
    blocks = [series[i*block_size:(i+1)*block_size].values for i in range(n_blocks)]
    
    # Handle remainder if exists
    remainder = n % block_size
    if remainder > 0:
        blocks.append(series[n_blocks*block_size:].values)
    
    np.random.shuffle(blocks)
    # Maintain original index structure
    shuffled_values = np.concatenate(blocks)
    return pd.Series(shuffled_values, index=series.index[:len(shuffled_values)])

def test_granger_causality(Y, X_base, df, n_lags=5, test_size=0.2, 
                           epochs=30, units=30, n_bootstrap=200, block_size=30):
    """
    Test Granger causality using LSTM neural networks with block bootstrapping.
    
    Parameters:
    - Y: Target variable (string)
    - X_base: Base predictor variable (string) 
    - df: DataFrame containing all variables and their lags
    - n_lags: Number of lags to use
    - test_size: Proportion of data for testing
    - epochs: Number of training epochs for LSTM
    - units: Number of LSTM units
    - n_bootstrap: Number of bootstrap iterations
    - block_size: Size of blocks for block bootstrapping
    
    Returns:
    - d_bar_original: Original test statistic
    - p_value: Bootstrap p-value
    - causal: Boolean indicating significant causality
    - bootstrap_stats: Array of bootstrap statistics
    """
    
    # Validate inputs
    if Y not in df.columns:
        raise ValueError(f"Target variable '{Y}' not found in DataFrame")
    if X_base not in df.columns:
        raise ValueError(f"Base predictor '{X_base}' not found in DataFrame")
    
    # Create lags for Y if they don't exist
    temp_df = pd.DataFrame(index=df.index)
    temp_df[Y] = df[Y]
    for i in range(1, n_lags+1):
        temp_df[f'{Y}_lag_{i}'] = temp_df[Y].shift(i)
    
    # Identify lag columns for X_base
    X_lags = [f'{X_base}_lag_{i}' for i in range(1, n_lags+1)]
    missing = [col for col in X_lags if col not in df.columns]
    if missing:
        print(f"Missing lag columns for {X_base}: {missing}")
        return None, None, False, None
    
    # Combine data and drop NA
    data = pd.concat([temp_df, df[X_lags]], axis=1).dropna()
    
    if len(data) < 50:  # Minimum data requirement
        print(f"Insufficient data after removing NAs: {len(data)} rows")
        return None, None, False, None
    
    y_data = data[Y].values
    Y_lag_cols = [f'{Y}_lag_{i}' for i in range(1, n_lags+1)]
    full_index = data.index
    
    # Time-based train-test split
    n_test = max(1, int(len(data) * test_size))  # Ensure at least 1 test sample
    n_train = len(data) - n_test
    
    if n_train < 10:  # Minimum training requirement
        print(f"Insufficient training data: {n_train} samples")
        return None, None, False, None
    
    # Build datasets
    X_restricted = data[Y_lag_cols].values
    X_unrestricted = np.zeros((len(data), n_lags, 2))
    
    for i in range(len(data)):
        for j in range(n_lags):
            X_unrestricted[i, j, 0] = data[Y_lag_cols[j]].iloc[i]
            X_unrestricted[i, j, 1] = data[X_lags[j]].iloc[i]
    
    # Split data
    X_res_train, X_res_test = X_restricted[:n_train], X_restricted[n_train:]
    X_unres_train, X_unres_test = X_unrestricted[:n_train], X_unrestricted[n_train:]
    y_train, y_test = y_data[:n_train], y_data[n_train:]
    
    # Scaling
    scaler_res = StandardScaler()
    X_res_train_scaled = scaler_res.fit_transform(X_res_train)
    X_res_test_scaled = scaler_res.transform(X_res_test)
    X_res_train_3d = X_res_train_scaled.reshape((-1, n_lags, 1))
    X_res_test_3d = X_res_test_scaled.reshape((-1, n_lags, 1))
    
    scaler_unres = StandardScaler()
    X_unres_train_2d = X_unres_train.reshape((-1, n_lags*2))
    X_unres_test_2d = X_unres_test.reshape((-1, n_lags*2))
    X_unres_train_scaled = scaler_unres.fit_transform(X_unres_train_2d)
    X_unres_test_scaled = scaler_unres.transform(X_unres_test_2d)
    X_unres_train_3d = X_unres_train_scaled.reshape((-1, n_lags, 2))
    X_unres_test_3d = X_unres_test_scaled.reshape((-1, n_lags, 2))
    
    # Model building function
    def build_model(input_shape):
        model = Sequential([
            LSTM(units, input_shape=input_shape, return_sequences=False),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])
        return model
    
    # Train original models with error handling
    try:
        model_res = build_model((n_lags, 1))
        model_unres = build_model((n_lags, 2))
        
        # Suppress verbose output
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            model_res.fit(X_res_train_3d, y_train, epochs=epochs, batch_size=min(32, len(y_train)//2), verbose=0)
            model_unres.fit(X_unres_train_3d, y_train, epochs=epochs, batch_size=min(32, len(y_train)//2), verbose=0)
        
    except Exception as e:
        print(f"Error training models: {str(e)}")
        return None, None, False, None
    
    # Original predictions and errors
    try:
        pred_res = model_res.predict(X_res_test_3d, verbose=0).flatten()
        pred_unres = model_unres.predict(X_unres_test_3d, verbose=0).flatten()
    except Exception as e:
        print(f"Error making predictions: {str(e)}")
        return None, None, False, None
    
    e_res = y_test - pred_res
    e_unres = y_test - pred_unres
    d_original = e_res**2 - e_unres**2
    d_bar_original = np.mean(d_original)
    
    # Prepare for bootstrapping
    base_series = df.loc[full_index, X_base].copy()
    bootstrap_stats = []
    
    print(f"Running {n_bootstrap} bootstrap iterations...")
    
    for i in range(n_bootstrap):
        if (i + 1) % 50 == 0:
            print(f"Bootstrap iteration {i + 1}/{n_bootstrap}")
        
        try:
            # Create shuffled X series with block bootstrapping
            shuffled_base = block_shuffle(base_series, block_size)
            
            # Create new lag features from shuffled series
            temp = pd.DataFrame(index=full_index)
            temp[X_base] = shuffled_base
            for j in range(1, n_lags+1):
                temp[f'{X_base}_lag_{j}'] = temp[X_base].shift(j)
            
            # Build new unrestricted dataset
            X_unrestricted_shuffled = np.zeros((len(data), n_lags, 2))
            for k in range(len(data)):
                for j in range(n_lags):
                    X_unrestricted_shuffled[k, j, 0] = data[Y_lag_cols[j]].iloc[k]
                    # Handle potential NaN values from shifted data
                    lag_val = temp[X_lags[j]].iloc[k] if k < len(temp) else 0
                    X_unrestricted_shuffled[k, j, 1] = lag_val if not pd.isna(lag_val) else 0
            
            # Extract test portion and scale
            X_unres_test_shuffled = X_unrestricted_shuffled[n_train:]
            X_unres_test_shuffled_2d = X_unres_test_shuffled.reshape((-1, n_lags*2))
            X_unres_test_shuffled_scaled = scaler_unres.transform(X_unres_test_shuffled_2d)
            X_unres_test_shuffled_3d = X_unres_test_shuffled_scaled.reshape((-1, n_lags, 2))
            
            # Predict with shuffled X
            pred_unres_shuffled = model_unres.predict(X_unres_test_shuffled_3d, verbose=0).flatten()
            e_unres_shuffled = y_test - pred_unres_shuffled
            
            # Compute loss difference
            d_shuffled = e_res**2 - e_unres_shuffled**2
            d_bar_shuffled = np.mean(d_shuffled)
            bootstrap_stats.append(d_bar_shuffled)
            
        except Exception as e:
            print(f"Error in bootstrap iteration {i}: {str(e)}")
            # Use a neutral value for failed iterations
            bootstrap_stats.append(0.0)
    
    # Calculate bootstrap p-value
    bootstrap_stats = np.array(bootstrap_stats)
    valid_stats = bootstrap_stats[~np.isnan(bootstrap_stats)]
    
    if len(valid_stats) < n_bootstrap * 0.5:  # If more than half failed
        print(f"Too many bootstrap failures: {n_bootstrap - len(valid_stats)}/{n_bootstrap}")
        return d_bar_original, None, False, bootstrap_stats
    
    # Conservative p-value calculation
    p_value = (np.sum(valid_stats >= d_bar_original) + 1) / (len(valid_stats) + 1)
    causal = (d_bar_original > 0) and (p_value < 0.05)
    
    return d_bar_original, p_value, causal, bootstrap_stats



In [ ]:
company = 'CCC'
df = companies_data_daily_final_full[companies_data_daily_final_full['company'] == company]
stationary_columns = [k for k,v in stationarity_results[company]['TWITTER'].items() if v]
stationary_columns_STOCK = [k for k,v in stationarity_results[company]['STOCK'].items() if v]
stationary_columns_with_lags = [x for x in df.columns if x.startswith(tuple(stationary_columns))]


lstm_result = {}
for company in companies:
    lstm_result[company] = {}
    stationary_columns = [k for k,v in stationarity_results[company]['TWITTER'].items() if v]
    stationary_columns_STOCK = [k for k,v in stationarity_results[company]['STOCK'].items() if v]
    df = companies_data_daily_final_full[companies_data_daily_final_full['company'] == company]
    
    for stock_col in stationary_columns_STOCK:
        lstm_result[company][stock_col] = {}
        
        for twitter_var in stationary_columns_with_lags:
           
            model_results = test_model_metrics(df=df,tweet_cols=[twitter_var],stock_col=[stock_col],company=company)
            lstm_result[company][stock_col][twitter_var] = model_results


In [ ]:
results = []
for col in stationary_columns:
    d_bar_original, p_value, causal, bootstrap_stats = test_granger_causality(
        Y='Wolumen',
        X_base='Positive',
        df=companies_data_daily_final_full[companies_data_daily_final_full['company'] == 'CCC'],
        n_lags=1,
        test_size=0.2,
        epochs=60,
        units=48,
        block_size= autocorrelation_results_ord['CCC']['TWITTER'][col],
        n_bootstrap=300
    )
    results.append({
        'Variable': col,
        'd_bar_original': d_bar_original,
        'p_value': p_value,
        'Causal': causal,
        'Bootstrap Stats': bootstrap_stats
    })

In [ ]:
# --- Permutation helpers ---
def _pick_block_len(T, p, base=None):
    rot = int(np.ceil(1.5 * T**(1/3)))  # rule-of-thumb
    b = max(3, p + 1, rot, base or 0)
    return min(b, max(3, T // 5))  # keep your original cap for simplicity

# Stationary bootstrap (Politis–Romano)
def _stationary_bootstrap(s: pd.Series, p_start: float, rng=None) -> pd.Series:
    """
    Bootstrap series with random-length circular blocks.
    p_start = 1 / expected_block_length.
    """
    rng = rng or np.random.default_rng()
    x = s.to_numpy()
    n = len(x)
    out = np.empty(n, dtype=x.dtype)
    pos = rng.integers(0, n)
    out[0] = x[pos]
    for t in range(1, n):
        pos = rng.integers(0, n) if rng.random() < p_start else (pos + 1) % n
        out[t] = x[pos]
    return pd.Series(out, index=s.index)

def granger_permutation(df, y, x, p, B=1000, base_block=None, exog=None):
    """
    Permutation-based Granger test:
      - Restricted model: y ~ lags(y) + exog
      - Unrestricted:     y ~ lags(y) + lags(x) + exog
    Returns dict with F, p_perm, B_valid, etc.
    """
    exog = exog or []

    # 1) Base frame: only the needed columns, no NaNs, sorted
    df = df[[y, x] + exog].dropna().copy().sort_index()

    # 2) Build lag columns
    for L in range(1, p + 1):
        df[f'{y}_L{L}'] = df[y].shift(L)
        df[f'{x}_L{L}'] = df[x].shift(L)

    cols_y = [f'{y}_L{L}' for L in range(1, p + 1)]
    cols_x = [f'{x}_L{L}' for L in range(1, p + 1)]

    # 3) Final modeling data (drops first p rows due to lags)
    data = df[[y] + cols_y + cols_x + exog].dropna()
    n = len(data)
    if n < 2 * p + 5:
        return {"ok": False, "reason": "too_few_obs"}

    Y = data[y].to_numpy()
    Xr = sm.add_constant(data[cols_y + exog], has_constant='add')
    Xu = sm.add_constant(data[cols_y + cols_x + exog], has_constant='add')

    m_r = sm.OLS(Y, Xr).fit()
    m_u = sm.OLS(Y, Xu).fit()

    rss_r, rss_u = m_r.ssr, m_u.ssr
    k = p  # number of added x-lag regressors
    dfd = m_u.df_resid  # robust degrees of freedom
    if dfd <= 0:
        return {"ok": False, "reason": "nonpos_df"}

    F_act = ((rss_r - rss_u) / k) / (rss_u / dfd)

    # 4) Bootstrap settings
    block = _pick_block_len(len(data), p, base_block)
    p_start = 1.0 / block
    idx = data.index  # rows kept after lagging

    # 5) Permutation loop (bootstrap x on full df index, then subset to idx)
    cnt = 0
    val = 0
    for _ in range(B):
        x_perm = _stationary_bootstrap(df[x], p_start)

        # Build x-lags on full index, then subset to idx
        xlags_full = pd.DataFrame(
            {f'{x}_L{L}': x_perm.shift(L) for L in range(1, p + 1)},
            index=df.index
        )

        Xb = pd.concat(
            [
                data[cols_y].reset_index(drop=True),
                xlags_full.loc[idx, cols_x].reset_index(drop=True),
                data[exog].reset_index(drop=True)
            ],
            axis=1
        )
        Xb = sm.add_constant(Xb, has_constant='add')

        # Should be NaN-free now; keep a light guard
        if Xb.isnull().any().any():
            continue

        rss_u_b = sm.OLS(Y, Xb).fit().ssr
        F_b = ((rss_r - rss_u_b) / k) / (rss_u_b / dfd)

        cnt += (F_b >= F_act)
        val += 1

    p_perm = (cnt + 1) / (val + 1) if val else np.nan
    return {
        "ok": True,
        "F": float(F_act),
        "p_perm": float(p_perm) if np.isfinite(p_perm) else np.nan,
        "B_valid": int(val),
        "block": int(block),
        "p": int(p),
    }

# ======================
# ====== MAIN RUN ======
# ======================
filename = "granger_permutation_results.csv"

if os.path.exists(filename):
    print("Loading existing file...")
    df_perm = pd.read_csv(filename)
else:
    B = 1000
    BASE_BLOCK = 5
    rows = []

    # Expecting these to be defined in your environment:
    # - stationary_columns: {company: {'STOCK': [...], 'TWITTER': [...]}, ...}
    # - companies_data_daily_final_full: DataFrame with at least ['company','Date', ...]
    for company, cols in stationary_columns.items():
        dfc = companies_data_daily_final_full.loc[
            companies_data_daily_final_full['company'] == company
        ].copy()
        dfc.index = pd.to_datetime(dfc['Date'])
        dfc = dfc.sort_index()

        for y in cols['STOCK']:
            for x in cols['TWITTER']:
                pair = dfc[[y, x]].dropna()
                T = len(pair)
                if T < 50:
                    continue

                p_cap = min(20, max(1, T // 8))
                try:
                    sel = VAR(pair).select_order(maxlags=p_cap)
                    p = int(max(1, sel.bic))
                except Exception:
                    p = 1

                if T < 8 * p:
                    continue

                res = granger_permutation(dfc, y=y, x=x, p=p, B=B, base_block=BASE_BLOCK)
                res.update({"T": int(T), "Company": company, "Target Variable": y, "Source Feature": x})
                if res.get("ok"):
                    rows.append({
                        "Company": company,
                        "Target Variable": y,
                        "Source Feature": x,
                        "Lag": res["p"],
                        "F_stat": res["F"],
                        "p_value_perm": res["p_perm"],
                        "B_valid": res["B_valid"],
                        "block": res["block"],
                        "T": res["T"],
                    })

    df_perm = pd.DataFrame(rows)
    df_perm.to_csv(filename, index=False)

# --- (Re)compute BH–FDR within (Company, Target Variable), safely handling NaNs ---
if not df_perm.empty and 'p_value_perm' in df_perm.columns:
    df_perm['p_value_perm_bh'] = np.nan
    df_perm['significant_perm_bh'] = False

    for (c, t), idx_grp in df_perm.groupby(['Company', 'Target Variable']).groups.items():
        mask = df_perm.index.isin(idx_grp)
        valid = mask & np.isfinite(df_perm['p_value_perm'])
        if valid.any():
            rej, p_adj, *_ = multipletests(
                df_perm.loc[valid, 'p_value_perm'], alpha=0.05, method='fdr_bh'
            )
            df_perm.loc[valid, 'p_value_perm_bh'] = p_adj
            df_perm.loc[valid, 'significant_perm_bh'] = rej

    df_perm['Flag'] = df_perm['significant_perm_bh'].fillna(False).astype(bool)

    # persist the updated BH/Flag columns
    df_perm.to_csv(filename, index=False)

print("Done.")

In [ ]:
res = test_granger_causality(
        Y='Wolumen',
        X_base='sentiment_divergence',
        df=companies_data_daily_final_full[companies_data_daily_final_full['company'] == 'CCC'],
        n_lags=1,
        test_size=0.2,
        block_size= autocorrelation_results_ord['CCC']['TWITTER']['sentiment_divergence'],
        n_bootstrap=100
    )

sns.histplot(res['boostrap_stats'], kde=True)  # kde=True adds smooth curve
plt.title("Distribution of Data")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# company = 'CCC'
# df = companies_data_daily_final_full[companies_data_daily_final_full['company'] == company]
# stationary_columns = [k for k,v in stationarity_results[company]['TWITTER'].items() if v]
# stationary_columns_STOCK = [k for k,v in stationarity_results[company]['STOCK'].items() if v]
# stationary_columns_with_lags = [x for x in df.columns if x.startswith(tuple(stationary_columns))]

In [ ]:
# import shap
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.model_selection import TimeSeriesSplit
# import numpy as np
# import pandas as pd
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from sklearn.preprocessing import StandardScaler
# from scipy.stats import norm, t
# import numpy as np
# import pandas as pd
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from sklearn.preprocessing import StandardScaler
# import warnings

In [ ]:
# Modelling precomputed lags turned out to be too complex, so we will not use it in the final analysis.
# companies_data_daily_final = {}
# for company, df in companies_data_daily.items():
# #     companies_data_daily_final[company] = create_prediction_lags(df, tweet_col=tweet_columns, max_lags=5)
# # companies_data_daily_final_full = pd.DataFrame()
# tweet_columns_with_lagged = [x for x in companies_data_daily_final['ALLEGRO'] if x.startswith(tuple(tweet_columns))]

In [ ]:
# Initialize results dictionaries
autocorrelation_results = {}
# Autocorrelation threshold
autocorr_p_threshold = 0.05  # p-value threshold for autocorrelation significance
max_lags = 10  # Maximum number of lags to test

# Check autocorrelation for each variable
for company in companies:
    STOCK = [k for k,v in stationarity_results[company]['STOCK'].items() if v]
    TWITTER = [k for k,v in stationarity_results[company]['TWITTER'].items() if v]
    autocorrelation_results[company] = {'STOCK': {}, 'TWITTER': {}}
    
    for variable in STOCK + TWITTER:
        # Extract data for the specific variable and company
        series = companies_data_daily_final_full[companies_data_daily_final_full['company'] == company][variable].dropna()
        
        # Skip if series is too short
        if len(series) <= max_lags:
            category = 'STOCK' if variable in STOCK else 'TWITTER'
            autocorrelation_results[company][category][variable] = False
            continue
        
        # Perform the Ljung-Box test for autocorrelation
        from statsmodels.stats.diagnostic import acorr_ljungbox
        result = acorr_ljungbox(series, lags=max_lags, return_df=True)
        
        # Check if any lag shows significant autocorrelation
        p_values = result['lb_pvalue']
        has_autocorrelation = any(p_values < autocorr_p_threshold)
        
        # Store whether the variable has significant autocorrelation
        category = 'STOCK' if variable in STOCK else 'TWITTER'
        autocorrelation_results[company][category][variable] = has_autocorrelation

pprint.pprint(autocorrelation_results)

In [ ]:
IC = "bic"
ALPHA = 0.05
MIN_T = 50

granger_results = {}

for company, _ in stationarity_results.items():
    granger_results[company] = {}

    df_company = companies_data_daily_final_full.loc[
        companies_data_daily_final_full["company"] == company
    ].copy()
    df_company.index = pd.to_datetime(df_company["Date"])
    df_company = df_company.sort_index()

    stationary_stock   = stationary_columns[company]["STOCK"]
    stationary_twitter = stationary_columns[company]["TWITTER"]

    for stock_var in stationary_stock:
        granger_results[company][stock_var] = {}

        for twitter_var in stationary_twitter:
            df_pair = df_company[[stock_var, twitter_var]].dropna().copy()
            T = len(df_pair)
            if T < MIN_T:
                continue

            model = VAR(df_pair)
            p_cap = min(20, max(1, T // 8))
            sel = model.select_order(maxlags=p_cap)
            p_opt = int(max(1, getattr(sel, IC)))   # AIC or BIC

            if T < 8 * p_opt:
                continue

            # --- FIT (avoid name 'var_results') ---
            res = VAR(df_pair).fit(maxlags=int(p_opt), ic=None, trend="c")
            assert res.k_ar == int(p_opt)  # sanity check

            # Granger: twitter_var -> stock_var
            gr = res.test_causality(caused=stock_var, causing=twitter_var, kind="f")

            # Stability & whiteness
            is_stable = bool(res.is_stable())
            try:
                wh = res.test_whiteness(nlags=min(10, max(5, p_opt + 2)))
                white_p = float(wh.pvalue)
                resid_white = bool(white_p >= ALPHA)
            except Exception:
                white_p, resid_white = np.nan, True

            # ARCH per equation
            diagnostics = {}
            resid = res.resid
            for col in df_pair.columns:
                try:
                    lm_stat, lm_p, f_stat, f_p = het_arch(
                        resid[col].to_numpy(), nlags=min(10, T // 10)
                    )
                    diagnostics[col] = {"arch_lm_p": float(lm_p), "arch_flag": bool(lm_p < ALPHA)}
                except Exception:
                    diagnostics[col] = {"arch_lm_p": np.nan, "arch_flag": None}

            granger_results[company][stock_var][twitter_var] = {
                "p_opt": int(p_opt),
                "T": int(T),
                "ic_used": IC,
                "F_stat": float(gr.test_statistic),
                "p_value": float(gr.pvalue),
                "significant": bool(gr.pvalue < ALPHA),
                "stable": is_stable,
                "resid_white": resid_white,
                "resid_white_p": white_p,
                "diagnostics": diagnostics,
            }


In [ ]:
# # global picture
# p = df_perm['p_value_perm'].dropna().values
# print("m=", p.size, "min p=", p.min() if p.size else None, "median=", np.median(p) if p.size else None)

# # BH per (Company, Target), q=0.10 for exploratory
# for (c,t), idx in df_perm.groupby(['Company','Target Variable']).groups.items():
#     mask = df_perm.index.isin(idx)
#     rej, p_adj, *_ = multipletests(df_perm.loc[mask,'p_value_perm'], alpha=0.05, method='fdr_bh')
#     df_perm.loc[mask,'p_value_perm_bh'] = p_adj
#     df_perm.loc[mask,'significant_perm_bh'] = rej.astype(bool)

# # plot-ready flag (exploratory)
# df_perm['Flag'] = df_perm['significant_perm_bh'].fillna(False)


In [ ]:
# # Dictionary of variables to drop per company
# columns_to_drop = {
#     '11BIT': ['view_count_Neutral','engagement_impact'],
#     'ALLEGRO': ['Neutral',''],
#     'CCC': ['view_weighted_polarity'],
#     'CDR': ['tweet_volume', 'view_weighted_polarity'],
#     'INPOST': ['tweet_volume', 'view_weighted_polarity'],
#     'XTB': ['tweet_volume', 'retweet_count_Positive', 'view_weighted_polarity'],
#     'MENTZEN': ['view_count_Positive', 'retweet_count_Positive', 'sentiment_score_views',
#                 'retweet_count_Neutral', 'total_retweet_count', 'sentiment_intensity', 'view_weighted_polarity'],
#     'ŻABKA': ['tweet_volume']
# }
# columns_to_drop

# corr_per_company = {}
# for company in companies:
#     print(company)
#     df = companies_data_daily_final_full[companies_data_daily_final_full['company'] == company]

#     stationary_columns_curr = [k for k,v in stationarity_results[company]['TWITTER'].items() if v]
#     cols_to_drop = columns_to_drop.get(company, [])
#     df_corr = df[stationary_columns_curr].corr()
#     df_corr.reset_index(inplace=True)
#     df_corr_melted = pd.melt(df_corr,'index')

#     df_corr_melted = df_corr_melted[~((df_corr_melted['index'].isin(cols_to_drop)) | (df_corr_melted['variable'].isin(cols_to_drop)))]
#     corr_per_company[company] = df_corr_melted[(df_corr_melted['value']>=0.9) & (df_corr_melted['value']<1) ]
#     pprint.pprint(corr_per_company[company])